In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# import libraries

from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import glob
import cv2
import random as rd
import copy
%matplotlib inline

In [4]:
# batch, classes, epochs, Image size.
batch_size = 32
num_classes = 10
epochs = 10
IMG_SIZE = 48

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [6]:
train_labels = pd.read_csv('/content/gdrive/MyDrive/es/trainLabels.csv')
train_labels

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile
...,...,...
49995,49996,bird
49996,49997,frog
49997,49998,truck
49998,49999,automobile


In [7]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      50000 non-null  int64 
 1   label   50000 non-null  object
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [8]:
train_labels['label'].unique()

array(['frog', 'truck', 'deer', 'automobile', 'bird', 'horse', 'ship',
       'cat', 'dog', 'airplane'], dtype=object)

In [9]:
LABELS = ['frog', 'truck', 'deer', 'automobile', 'bird', 'horse', 'ship',
          'cat', 'dog', 'airplane']

In [10]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
y_train shape: (50000, 1)
y_test shape: (10000, 1)


In [11]:
# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [12]:
print(y_train[:10])

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [13]:
# convert to float, normalise the data

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.0
x_test /= 255.0

###Basic Model

In [14]:
# CNN Basic model architecture 

basic_model = Sequential()
basic_model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
basic_model.add(Activation('relu'))
basic_model.add(Conv2D(32, (3, 3)))
basic_model.add(Activation('relu'))
basic_model.add(MaxPooling2D(pool_size=(2, 2)))
basic_model.add(Dropout(0.25))

basic_model.add(Conv2D(64, (3, 3), padding='same'))
basic_model.add(Activation('relu'))
basic_model.add(Conv2D(64, (3, 3)))
basic_model.add(Activation('relu'))
basic_model.add(MaxPooling2D(pool_size=(2, 2)))
basic_model.add(Dropout(0.25))

basic_model.add(Flatten())
basic_model.add(Dense(512))
basic_model.add(Activation('relu'))
basic_model.add(Dropout(0.5))
basic_model.add(Dense(num_classes))
basic_model.add(Activation('softmax'))

In [ ]:
# summary
basic_model.summary()

In [ ]:
# compile the model
basic_model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
# train 
basic_model_history = basic_model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

###Batch model

In [15]:
batch_model = Sequential()
batch_model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
batch_model.add(Activation('relu'))
batch_model.add(BatchNormalization())
batch_model.add(Conv2D(32, (3, 3)))
batch_model.add(Activation('relu'))
batch_model.add(BatchNormalization())
batch_model.add(MaxPooling2D(pool_size=(2, 2)))

batch_model.add(Conv2D(64, (3, 3), padding='same'))
batch_model.add(Activation('relu'))
batch_model.add(BatchNormalization())
batch_model.add(Conv2D(64, (3, 3)))
batch_model.add(Activation('relu'))
batch_model.add(BatchNormalization())
batch_model.add(MaxPooling2D(pool_size=(2, 2)))

batch_model.add(Flatten())
batch_model.add(Dense(512))
batch_model.add(Activation('relu'))
batch_model.add(Dropout(0.5))
batch_model.add(Dense(num_classes))
batch_model.add(Activation('softmax'))

In [16]:
# summary of the model
print(batch_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation_6 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_5 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_7 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 30, 30, 32)       128       
 hNormalization)                                      

In [17]:
# compile
batch_model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [18]:
# Training the model
batch_model_history = batch_model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Epoch 1/10
1563/1563 [==============================] - 330s 210ms/step - loss: 1.5575 - accuracy: 0.4546 - val_loss: 1.2605 - val_accuracy: 0.5471
Epoch 2/10
1563/1563 [==============================] - 314s 201ms/step - loss: 1.1425 - accuracy: 0.5911 - val_loss: 1.1575 - val_accuracy: 0.6080
Epoch 3/10
1563/1563 [==============================] - 309s 198ms/step - loss: 0.9803 - accuracy: 0.6543 - val_loss: 0.9024 - val_accuracy: 0.6774
Epoch 4/10
1563/1563 [==============================] - 314s 201ms/step - loss: 0.8656 - accuracy: 0.6939 - val_loss: 0.8091 - val_accuracy: 0.7155
Epoch 5/10
1563/1563 [==============================] - 315s 201ms/step - loss: 0.7804 - accuracy: 0.7254 - val_loss: 0.7882 - val_accuracy: 0.7269
Epoch 6/10
1563/1563 [==============================] - 310s 198ms/step - loss: 0.7104 - accuracy: 0.7510 - val_loss: 0.7683 - val_accuracy: 0.7293
Epoch 7/10
1563/1563 [==============================] - 309s 198ms/step - loss: 0.6621 - accuracy: 0.7671 - val_

In [19]:
batch_model.save('my_model')

In [20]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_saved_model('my_model')
tflite_model = converter.convert()

# Save the TFLite model
with open('model2.tflite', 'wb') as f:
    f.write(tflite_model)